<a href="https://colab.research.google.com/github/laurence-lin/Kaggle_competition/blob/master/Ashrae_predict_FE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import pandas as pd
import math

import sklearn
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import os

import gc
from google.colab import files
# load data from Cloud Storage
from google.colab import auth
auth.authenticate_user()

# Configure GCP project and use gsutil to copy the file from storage

!gcloud config set project 'blind-detection'
!gsutil -m cp -r gs://ashare_dataset/*.csv  sample_data/



Updated property [core/project].
Copying gs://ashare_dataset/building_metadata.csv...
Copying gs://ashare_dataset/sample_submission.csv...
Copying gs://ashare_dataset/test.csv...
Copying gs://ashare_dataset/weather_test.csv...
Copying gs://ashare_dataset/train.csv...
Copying gs://ashare_dataset/weather_train.csv...
Resuming download for sample_data/test.csv component 0
Resuming download for sample_data/sample_submission.csv component 2
Resuming download for sample_data/train.csv component 1
Resuming download for sample_data/test.csv component 1
Resuming download for sample_data/sample_submission.csv component 0
Resuming download for sample_data/train.csv component 0
Resuming download for sample_data/test.csv component 2
Resuming download for sample_data/sample_submission.csv component 1
Resuming download for sample_data/train.csv component 2
Resuming download for sample_data/test.csv component 3
Resuming download for sample_data/train.csv component 3
\ [6/6 files][  2.4 GiB/  2.4 GiB] 

In [34]:
# Reduce memory function

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16


from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df


# Load all datasets and reduce memory
print(os.listdir('sample_data/'))
data_path = 'sample_data/'
train = pd.read_csv(os.path.join(data_path, 'train.csv'), parse_dates = ['timestamp'])
test = pd.read_csv(os.path.join(data_path, 'test.csv'), parse_dates = ['timestamp'])
building = pd.read_csv(os.path.join(data_path, 'building_metadata.csv'))
weather_test = pd.read_csv(os.path.join(data_path, 'weather_test.csv'), parse_dates = ['timestamp'])
#submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
weather_train = pd.read_csv(os.path.join(data_path, 'weather_train.csv'), parse_dates = ['timestamp'])

train = reduce_mem_usage(train, use_float16 = True)
building = reduce_mem_usage(building, use_float16 = True)
weather_train = reduce_mem_usage(weather_train, use_float16 = True)
test = reduce_mem_usage(test)
weather_test = reduce_mem_usage(weather_test)


['anscombe.json', 'README.md', 'weather_train.csv', 'train.csv', 'sample_submission.csv', 'building_metadata.csv', 'test.csv', 'weather_test.csv', 'california_housing_test.csv', 'mnist_train_small.csv', 'california_housing_train.csv', 'mnist_test.csv']
Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.1%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 3.07 MB
Decreased by 68.1%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 596.49 MB
Decreased by 53.1%
Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 9.78 MB
Decreased by 48.6%


1. Load dataset 
2. Do EDA to analyze data structure 
3. Do feature engineering 
4. Apply model training 
5. Make test data prediction

In [35]:
# Do interpolation for weather data first, for too much missing values. There may still be some missing values after this.
# Interpolate by each site across the timestamp
weather_train = weather_train.groupby('site_id').apply(lambda x_site: x_site.interpolate(limit_direction = 'both'))
weather_test = weather_test.groupby('site_id').apply(lambda x_site: x_site.interpolate(limit_direction = 'both'))

print('Missing values in weather train data after interpolation: \n')
print(weather_train.isnull().sum().sort_values(ascending = False))

Missing values in weather train data after interpolation: 

precip_depth_1_hr     26273
cloud_coverage        17228
sea_level_pressure     8755
wind_speed                0
wind_direction            0
dew_temperature           0
air_temperature           0
timestamp                 0
site_id                   0
dtype: int64


I can see that although some NaN values is filled in weather_data, interpolation couldn't fill all of the missing values.  
We will continue this in FE.

In [0]:
# Combine data and meta datas for advanced analysis
train = train.merge(building, on = 'building_id', how = 'left')
train = train.merge(weather_train, on = ['site_id', 'timestamp'], how = 'left')
test = test.merge(building, on = 'building_id', how = 'left')
test = test.merge(weather_test, on = ['site_id', 'timestamp'], how = 'left')

del building, weather_train, weather_test



In [0]:
# After the timestamp alignment, new columns 'offset' may generate whole row that is NaN. Drop these rows
# Which means whole row with all NaN values
weather_col = ['air_temperature', 'dew_temperature', 'wind_speed', \
               'wind_direction', 'offset', 'sea_level_pressure',
               'precip_depth_1_hr', 'cloud_coverage']
train = train.dropna(subset = weather_col, how = 'all')


In [39]:
# Fill NaNs
null_col = 100 - train.count()/len(train)*100
null_col = train.loc[:, null_col > 0] # dataframe from train that contain null columns

def mean_without_overflow_fast(col):
    # Compute mean value of each column which contains missing value
    col /= len(col)
    return col.mean() * len(col)

null_col_mean = null_col.apply(mean_without_overflow_fast)

for col in null_col.keys():
  # Fill mean with integer for year_built and floor_count
   if col == 'year_built' or col =='floor_count':
     train[col].fillna(math.floor(null_col_mean[col]), inplace = True)
     test[col].fillna(math.floor(null_col_mean[col]), inplace = True)
   else:
     train[col].fillna(null_col_mean[col], inplace = True)
     test[col].fillna(null_col_mean[col], inplace = True)



/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [40]:
# Create new feature: feature generation

def time_transform(df):
  df['hour'] = df['timestamp'].dt.hour
  df['year'] = df['timestamp'].dt.year
  df['month'] = df.timestamp.dt.month
  df['day'] = df.timestamp.dt.day
  df['dayofweek'] = df.timestamp.dt.dayofweek
  
  return df

train = time_transform(train)
test = time_transform(test)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [0]:
# Create is_holiday feature by US_Holiday calendar
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# Add is_holiday = 1: The date that within USA_holiday, and the dates that is weekend

date_range = pd.date_range(start = train['timestamp'].min(), end = test['timestamp'].max())
us_holidays = calendar().holidays(start = date_range.min(), end = date_range.max()) # USA holidays within data date_range
# Only datetime64 could apply isin() function, which is convenient
train['is_holiday'] = (train['timestamp'].dt.date).astype('datetime64').isin(us_holidays).astype(np.int8)
test['is_holiday'] = (test['timestamp'].dt.date).astype('datetime64').isin(us_holidays).astype(np.int8)

train.loc[(train['timestamp'].dt.dayofweek == 5) | (train['timestamp'].dt.dayofweek == 6), 'is_holiday'] = 1
test.loc[(test['timestamp'].dt.dayofweek == 5) | (test['timestamp'].dt.dayofweek == 6), 'is_holiday'] = 1


gc.collect()


In [0]:
# Encode cyclic features
def encode_cyclic_feature(df, col, max_val):
  '''
  Encode cyclic feature with sin cosine transform
  df: dataframe contains cyclic feature
  col: cylcic features to transform
  max_val: max value for that cyclic column
  '''
  df[col + '_sin'] = np.sin(2*np.pi*(df[col]/max_val))
  del df[col]
  return df

train = encode_cyclic_feature(train, 'dayofweek', 7)
train = encode_cyclic_feature(train, 'hour', 24)
train = encode_cyclic_feature(train, 'day', 31)
train = encode_cyclic_feature(train, 'month', 31)

In [0]:
# Data cleaning: drop useless features and samples
train.corr()['meter_reading'].plot()

drop_features = ['wind_speed', 'sea_level_pressure']

gc.collect()

In [0]:
# Feature engineering

def preprocessing(df, building, weather, is_train = False):
  '''
  df: train data or test dataframe
  '''

  # 1. Log transformation for training target
  if is_train:
    df['meter_reading'] = np.log1p(df['meter_reading'])
  
  df['square_feet'] = np.log1p(df['square_feet'])

  # 2. Fill NaNs
  

  # 3. Categorical encoding

  # 4. Data cleaning: NaN rows, Outliers, ...etc




  return df